In [2]:
from IPython.display import display, HTML
display(HTML("""<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{fontsize:12pt;padding:5px;}
table.dataframe{font-size:12px;}))
</style>
"""))

# <span style="color:red">ch09 06 LangChain과 vectorDatabase를 활용한 RAG구현 </span>

[ RAG 구현 절차 ]

```
1. 문서의 내용을 읽는다 (document_loader를 이용)
    1) https://python.langchain.com/v0.2/docs/integrations/document_loaders/ 
    2) https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/
    %pip install --upgrade --quiet docx2txt
2. 문서를 쪼갠다 (한번에 이해하고 처리할 수 있는 입력 + 출력 토큰수가 제한)
    1) https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries 
    %pip install -qU langchain-text-splitters
3. 쪼갠 문서를 임베딩하여 vector database에 넣음
    1) OpenAIEmbeddings나 UpstageEmbeddings 이용해서 임베딩
    2) https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/  
    %pip install –q langchain-chroma
4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문으로 전달하여 답변 얻음 (제공되는 Prompt활용)
    1) https://python.langchain.com/v0.2/docs/tutorials/rag/
    %pip install –q langchain langchainhub
    http://smith.langchain.com에서 key생성 .env key(LANGCHAIN_API_KEY) 추가
```

# 0. 패키지 설치

In [3]:
# 문서 읽어오기
%pip install --quiet docx2txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
# 텍스트를 청크로 나누는 기능만 있는 경량 모듈
%pip install -q langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [5]:
# 벡터 DB (로컬 DB)
%pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [6]:
# 제공되는 prompt 사용
%pip install -q langchain langchainhub

Note: you may need to restart the kernel to use updated packages.


# 1. 문서 읽기 (X)

In [26]:
%%time

from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
document = loader.load()

CPU times: total: 3.91 s
Wall time: 3.9 s


In [27]:
len(document)

1

In [28]:
document[0].page_content[:200]

'소득세법\n\n소득세법\n\n[시행 2025. 7. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]\n\n기획재정부(재산세제과(양도소득세)) 044-215-4312\n\n기획재정부(소득세제과(근로소득)) 044-215-4216\n\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233\n\n기획재정부(소득세제과(사업소득, 기타소득)) 044-2'

# 2. 문서를 쪼개면서 읽기 (o)

In [29]:
import time
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

start = time.time()
loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
text_splitter = RecursiveCharacterTextSplitter(  # 문서를 쪼개는 기준이 문자수
    chunk_size = 1500,  # 문서를 쪼갤 때 1500글자씩
    chunk_overlap = 200
)
# 1번째 chunk 1 ~ 1450글자
# 2번째 chunk 1250 ~ 1750글자
document = loader.load_and_split(text_splitter = text_splitter)
runtime = time.time() - start
print('문서 쪼개면서 읽는 시간 :', runtime)

문서 쪼개면서 읽는 시간 : 4.100828647613525


In [30]:
# chunk 갯수

len(document)

183

In [31]:
len(document[0].page_content)

1464

In [32]:
# chunk의 글자수들

# [len(doc.page_content) for doc in document]
print(max(len(doc.page_content) for doc in document))
print(min(len(doc.page_content) for doc in document))

1497
1055


# 3. 쪼갠 문서를 임베딩 → 벡터 데이터베이스 저장

- 임베딩 모델 : openAI API의 text-embedding-3-large (기본 : text-embedding-ada-002)
- 벡터 데이터베이스 : chroma

In [33]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
# https://python.langchain.com/v0.2/docs/how_to/embed_text/
embedding = OpenAIEmbeddings(
    model = "text-embedding-3-large"
)

In [34]:
embeddings = embedding.embed_documents(
    [
        "소득세법 어쩌구 저쩌구",
        document[0].page_content
    ]
)
len(embeddings), len(embeddings[0])

(2, 3072)

In [35]:
len(embeddings), len(embeddings[0]), len(embeddings[1])

(2, 3072, 3072)

In [36]:
len(embedding.embed_query("소득세"))

3072

In [37]:
%%time

from langchain_chroma import Chroma

# 데이터를 처음 저장할 때
# database = Chroma.from_documents(                                 
#     documents = document,
#     embedding = embedding,
#     collection_name = "tax-collection",  # 생략 시 이름 랜덤
#     persist_directory = './chroma'  # 생략 시 로컬데이터베이스에 저장 안됨, 프로그램 종료시 db날라감
# )
# 이미 저장된 vector DB를 사용할 때
database = Chroma(
    embedding_function = embedding,
    collection_name = "tax-collection",
    persist_directory = './chroma'
)

CPU times: total: 31.2 ms
Wall time: 32.4 ms


# 4. vector DB에 질문과 유사도 검색 (답변 생성을 위한 retrieval)

In [38]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query,
                                           k = 3)  # 기본 k는 4

In [39]:
retrieved_docs

[Document(id='d2f774e3-1ad2-4bf1-8697-cb93e02bad6d', metadata={'source': './tax_docs/소득세법(법률)(제20615호)(20250701).docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM 전달하여 답변 생성

In [40]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4.1-nano")

In [41]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문자입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요
[context]는 다음과 같아요
{retrieved_docs}
Question : {query}"""

In [42]:
ai_message = llm.invoke(prompt)

In [43]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 먼저 과세표준과 세율을 적용해야 합니다. 

2025년 기준 국내의 근로소득에 대한 기본 세율 구조는 아래와 같습니다(일반적인 세율 표는 참고용입니다; 세율은 매년 조정될 수 있으니 정확한 연도 세율표를 확인하는 것이 좋습니다).

| 과세표준 구간 | 세율 | 누진공제액 |
|----------------|-------|--------------|
| 1,200만원 이하 | 6% | 0 |
| 1,200만원 초과 4,600만원 이하 | 15% | 108만원 |
| 4,600만원 초과 8,800만원 이하 | 24% | 522만원 |
| 8,800만원 초과 | 35% | 1,490만원 |

먼저, 연봉이 50,000,000원인 경우 공제액(근로소득공제)을 차감해야 합니다. 

(참고: 2025년 기준 근로소득공제율은 다소 조정될 수 있으며, 일반적으로 일정액 또는 일정비율 공제 방식을 따릅니다. 예를 들어, 연봉 5,000만원 기준 근로소득공제는 대략 1,500만원 정도 추정됩니다.)

**계산 과정:**

1. **근로소득공제 적용 후 과세표준:**  
50,000,000원 - 15,000,000원 (추정 공제액) ≈ 35,000,000원

2. **세율 구간별 세금 계산:**  
- 1,200만원까지: 12,000,000원 × 6% = 720,000원  
- 1,200만원 초과 4,600만원 이하 구간: (35,000,000 - 12,000,000) = 23,000,000원  
  세율 15% 적용: 23,000,000 × 15% = 3,450,000원  
- 4,600만원 초과 구간(이 경우는 없음, 35억 이하이므로 이 단계는 생략)

그러나 앞서 계산은 간단화를 위해 3단계로 나누어 계산하고, 누진공제액도 반영해야 정확합니다. 

**최종 세액 계산 (대략):**

세액 = (12,000,000 × 6%) + (23,000,000 × 15%) - 누진공제액  
= 720,000 + 3,450,000 -

# 6. Augmentation을 위한 제공되는 Prompt활용하여 langchain으로 답변 생성

In [44]:
from langchain import hub

query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

prompt = hub.pull("rlm/rag-prompt")
prompt

C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

## RetrievalQA를 통해 LLM전달 (create_retrieval_chain이 대체)

```
 query → retriever 전달 (백터 검색 수행) 
 → retrieval문서 → prompt의 {context}에 삽입
 → query → prompt의 {question}에 삽입
```

In [45]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(search_kwargs = {'k':5}),
    chain_type_kwargs = {"prompt":prompt}
)

In [46]:
ai_message = qa_chain.invoke({"query":query})

In [47]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 구체적인 세율과 공제 조건에 따라 달라지므로 이 자료만으로 정확한 금액을 알 수 없습니다. 일반적으로 소득세는 종합소득과 세율, 공제액 등을 고려하여 계산됩니다. 따라서 정확한 세액을 알기 위해서는 세금 계산 프로그램 또는 세무 전문가의 상담이 필요합니다.'}